# Tutorial dla Tweepy

## Podstawowe możliwosci

### Instalacja i uwierzytelnienie

Pobieramy najnowszą wersję.

Aby się uwierzytelnić używamy naszego tokenu z developer.twitter.com

Poniższy został przyznany w ramach grantu akademickiego, pozwala na pobranie 10 milionow Twittow miesięcznie i korzystać z wszystkich fukcjonalnosć Twittera API (przede wszystkim z wyszukiwania Twittow starszych niż tydzień).

In [5]:
academic_bearer = "AAAAAAAAAAAAAAAAAAAAADIEawEAAAAAxzzD4cQ2g8FGK2%2BkKz2%2FJvTnoMA%3D09uegYs5HrQvrsFkAEl3WwxhspBYFBIH3Vnykec79asqiUsSoA"

In [13]:
import tweepy 
import pandas as pd
import re
import json

In [14]:
streaming = tweepy.StreamingClient(academic_bearer)

In [15]:
!pip install pyspark

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

spark = SparkSession \
        .builder \
        .appName('Test_threading') \
        .getOrCreate()
RDD = spark.sparkContext.emptyRDD()
schema = StructType([
  StructField('text', StringType(), True),
  StructField('tags', StringType(), True)
  ])
df = spark.createDataFrame(RDD,schema)

### Wyszukiwanie Tweetow

W przypadku normalnego dostępu posługujemy się funkcją, ktora znajduje tweety z ostatniego tygodnia:

In [26]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

class IDPrinter(tweepy.StreamingClient):
    def __init__(self,academic_bearer):
      super().__init__(academic_bearer)

    def on_data(self, data):
      msg = json.loads( data )
      tags =[]
      #print(msg['data']['text'])
      tt = []
      text = msg['data']['text']
      global emoji_pattern
      tt.append(emoji_pattern.sub(r'', text))
      for rule in msg['matching_rules']:
        if(rule['id'] == "1616159160550178822"):
          tags.append('biden')
        else:
          tags.append('trump')
      print(str("_".join(tags)))
      tt.append(str("_".join(tags)))
      global spark
      global schema
      global df
      newRow = spark.createDataFrame([tt], schema)
      df = df.union(newRow)





In [27]:
printer = IDPrinter("AAAAAAAAAAAAAAAAAAAAADIEawEAAAAAxzzD4cQ2g8FGK2%2BkKz2%2FJvTnoMA%3D09uegYs5HrQvrsFkAEl3WwxhspBYFBIH3Vnykec79asqiUsSoA")
printer.add_rules(tweepy.StreamRule("trump -is:reply -is:retweet -has:links lang:en"))
printer.add_rules(tweepy.StreamRule("biden -is:reply -is:retweet -has:links lang:en"))
printer.get_rules()

Response(data=[StreamRule(value='trump -is:reply -is:retweet -has:links lang:en', tag=None, id='1616159155525402629'), StreamRule(value='biden -is:reply -is:retweet -has:links lang:en', tag=None, id='1616159160550178822')], includes={}, errors=[], meta={'sent': '2023-01-19T23:28:24.642Z', 'result_count': 2})

In [28]:
printer.filter()

trump


KeyboardInterrupt: 

In [29]:
df.show()

+--------------------+-----+
|                text| tags|
+--------------------+-----+
|Actually, the deb...|trump|
+--------------------+-----+

